In [12]:
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import RidgeCV
from models.modelSurprise import computeSurprise
from models.modelNN import computeNN
from sklearn.model_selection import KFold
from helpers import create_csv, load_data

In [14]:
samples = load_data('data/sampleSubmission.csv')
data = load_data('data/data_train.csv')

In [ ]:
surprise_ratings = computeSurprise(data, samples)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
NN_ratings = computeNN(data, samples)

In [10]:
test_pred = pd.read_pickle('data/test_pred.pickle')
test_pred = test_pred.rename(columns={"mf_rmse_rating": "MF_RMSE_rating"})

In [11]:
models_names = [ 'MF_ALS_rating',
                'cocluster_rating',
                'knnmeans_item_rating',
                'knnmeans_user_rating',
                'knnzscore_user_rating',
                'knnzscore_item_rating',
                'knnbasic_user_rating',
                'knnbasic_item_rating',
                'slopeone_rating',
                'mf_rating',
                'svd_rating',
                'svdpp_rating',
                'item_mean_rating',
                'user_mean_rating',
                'global_mean_rating',
                'MF_RMSE_rating',
                'NN_deep_rating',
                'NN_shallow_rating']

cv_ridge = KFold(n_splits=10, random_state = 200)
gs_ridge = RidgeCV(alphas = [10**-i for i in range (-5, 10)], fit_intercept = False, scoring = 'neg_mean_squared_error', cv = cv_ridge)

/home/emma-hoggett/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [12]:
gs_ridge.fit (test_pred[models_names],test_pred['rating'] )

print ("Best lambda :", gs_ridge.alpha_, "\n")
print ("Optimal weight:", gs_ridge.coef_, "\n")

Best lambda : 1.0 

Optimal weight: [ 0.40679372 -0.00439563  0.06831155  0.2343497   0.18813534  0.10614723
  0.14039408  0.09197466  0.19605285 -0.04170352  0.01601419  0.18747773
 -0.59894364 -0.65511378  0.59057697 -0.21071062  0.16800608  0.12941344] 



In [13]:
submission_pred['ridge_rating'] = gs_ridge.predict(submission_pred[models_names])

30458	PaillasonLover	graded	1.017	0.172	Graded Successfully !

In [14]:
submission_path = 'FinalSubmission.csv'
create_csv(submission_path, submission_pred)

/home/emma-hoggett/git_workspace/Project2_ML/helpers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  final['Prediction'] = final['ridge_rating'].apply(lambda x: 5 if x > 5 else (1 if x < 1 else round(x)))
/home/emma-hoggett/git_workspace/Project2_ML/helpers.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  final['Id'] = pos
